In [ ]:
#AI Trainer for the Adjustments Project


In [4]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,Fix_Broken_Batch,CustomerID,727741,167749,0.15,CUST712035,Retail,High,0,US,CUST981372
1,Amend,Default,RiskCategory,322590,409969,0.15,CUST450896,SME,Low,0,DE,Medium
2,Amend,Defaulting_Reclassify,RiskCategory,916339,631088,0.95,CUST712035,Retail,Medium,1,GB,High
3,Amend,Defaulting_Reclassify,IsDefaulted,474573,963829,0.50,CUST192654,Corporate,Medium,1,DE,0
4,Amend,Market Downturn,CustomerType,402089,742127,0.12,CUST192654,Corporate,High,1,DE,0.75
5,Amend,Credit_Uplift,IsDefaulted,782759,987356,0.58,CUST238947,SME,High,0,DE,1
6,Amend,Default,IsDefaulted,483758,14262,0.62,CUST981372,Retail,High,0,GB,0
7,Amend,Credit_Uplift,IsDefaulted,663881,337374,0.26,CUST192654,SME,High,1,DE,0
8,Amend,Default,CustomerID,244540,45593,0.57,CUST192654,Retail,High,0,GB,CUST103289
9,Amend,Credit_Uplift,RiskCategory,231428,541960,0.47,CUST563201,Corporate,Low,1,GB,Low


In [6]:
#Stage 1: OneHot encode string inputs and labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)

#Turn output labels into numbers
outputLabels=df["NewVal"].unique()
# #Create a dictionary to map the output labels to numbers
outputLabelsDict={}
for i in range(len(outputLabels)):
    outputLabelsDict[outputLabels[i]]=i
df["NewVal"]=df["NewVal"].map(outputLabelsDict)

df_encoded=pd.get_dummies(df, columns=string_columns)

#Move NewVal to the end of the dataframe
df_encoded=df_encoded[[c for c in df_encoded if c not in ['NewVal']] + ['NewVal']]

df_encoded


,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,727741,167749,0.15,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,1,0
1,322590,409969,0.15,0,1,0,1,0,0,0,...,0,0,1,0,1,0,1,0,0,1
2,916339,631088,0.95,1,1,0,0,1,0,0,...,0,1,0,0,0,1,0,1,0,2
3,474573,963829,0.50,1,1,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,3
4,402089,742127,0.12,1,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,499655,799747,0.78,1,1,0,1,0,0,0,...,1,0,0,0,0,1,1,0,0,17
996,843487,320775,0.74,0,1,0,1,0,0,0,...,0,0,1,1,0,0,0,0,1,3
997,68954,745314,0.00,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,19
998,249039,355545,0.90,0,1,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,20


In [7]:
#Detach the NewVal column and onehot encode it!
labelF=df_encoded["NewVal"]
df_encoded=df_encoded.drop(columns=["NewVal"])

labelF=pd.get_dummies(labelF)
labelF

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
996,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe

#Normalize Drawn (-1 to 1)
df_normalized=df_encoded.copy()
df_normalized["Drawn"]=(df_encoded["Drawn"]-df_encoded["Drawn"].min())/(df_encoded["Drawn"].max()-df_encoded["Drawn"].min())*2-1
#Normalize Undrawn (-1 to 1)
df_normalized["Undrawn"]=(df_encoded["Undrawn"]-df_encoded["Undrawn"].min())/(df_encoded["Undrawn"].max()-df_encoded["Undrawn"].min())*2-1

#Clip PD to 0-1
df_normalized["PD"]=df_normalized["PD"].clip(0,1)


df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerID_CUST981372,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US
0,0.454850,-0.664079,0.15,0,1,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1
1,-0.356395,-0.178864,0.15,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,1,0,0
2,0.832485,0.264082,0.95,1,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
3,-0.052076,0.930629,0.50,1,1,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
4,-0.197212,0.486515,0.12,1,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.001853,0.601940,0.78,1,1,0,1,0,0,0,...,0,1,0,0,0,0,1,1,0,0
996,0.686612,-0.357537,0.74,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
997,-0.864258,0.492900,0.00,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1
998,-0.503669,-0.287886,0.90,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0


In [9]:
print(len(df_normalized))
print(len(labelF))

1000
1000


In [10]:
#Turn the pandas dataframe into a numpy array, split out the label and then break into sets
import tensorflow as tf

#turn df_normalized into a normal array
data=df_normalized.values

#Get all columns except the output column
X=data[:,1:]

#Get the output column
data=labelF.values
y=data[:,:]


from sklearn.model_selection import train_test_split
#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




In [11]:
print(len(X))
print(len(y))

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))


1000
1000
800
200
800
200


In [12]:
#Time to build the model

#Simple feed-forward model. No U-nets or anything fancy
ann = tf.keras.models.Sequential()

neurons=1000
num_classes=22

#Input and first hidden layer
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

#Output layer
#Classifier head
ann.add(tf.keras.layers.Dense(units=num_classes, activation='sigmoid'))
# ann.add(tf.keras.layers.Flatten())
# ann.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


#Compile!
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

ann.build(input_shape=(None, X_train.shape[1]))
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              35000     
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 22)                22022     
                                                                 
Total params: 2,059,022
Trainable params: 2,059,022
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Train the MODEL!!!!!


#y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))


ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

ValueError: Data cardinality is ambiguous:
  x sizes: 800
  y sizes: 17600
Make sure all arrays contain the same number of samples.